In [ ]:
pip install sagemaker -U

In [ ]:
import sagemaker
print(sagemaker.__version__)

In [ ]:
!pygmentize ./src/1/hello_sagemaker_training.py

In [ ]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
from sagemaker.pytorch import PyTorch
estimator = PyTorch(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    py_version='py38', 
    framework_version='1.9.1',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
from sagemaker.mxnet import MXNet
estimator = MXNet(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    py_version='py37', 
    framework_version='1.8.0',
    instance_count=1,
    instance_type='ml.m4.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    py_version='py3', 
    framework_version='0.23-1',
    instance_count=1,
    instance_type='ml.c5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
estimator.latest_training_job.describe()

In [ ]:
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    entry_point='./src/1-1/hello_sagemaker_training.py',
    image_uri = estimator.latest_training_job.describe()['AlgorithmSpecification']['TrainingImage'],
    instance_count=1,
    instance_type='ml.c5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit()

In [ ]:
cat ./data/1-2/calc.txt

In [ ]:
with open('./data/1-2/calc.txt','rt') as f:
    input_text_lines = f.read()
for input_text in input_text_lines.split('\n'):
    print(eval(input_text))

In [ ]:
import sagemaker
input_s3_uri = sagemaker.session.Session().upload_data(path='./data/1-2/calc.txt', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/1-2')
print(input_s3_uri)

In [ ]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./src/1-2/calc.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit(input_s3_uri)

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow
input_s3_uri = sagemaker.session.Session().upload_data(path='./data/1-3/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='training/1-3')
print(input_s3_uri)
estimator = TensorFlow(
    entry_point='./src/1-3/calc.py',
    py_version='py38', 
    framework_version='2.6.0',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit(input_s3_uri)